# Lucas exploration

## Libraries

In [3]:
import pandas as pd

## Data manipulation

In [4]:
df = pd.read_csv('./Manuel/full_lucas_fixed.csv')
df.sample(3)

/var/folders/ww/1lqc8cld6w36nm_7m2nv8sv00000gn/T/ipykernel_19930/4165562859.py:1: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./Manuel/full_lucas_fixed.csv')


,POINT_ID,geometry,NUTS0,NUTS1,NUTS2,NUTS3,TH_LAT,TH_LONG,OFFICE_PI,SURVEY_DATE,...,Ox_Al,Ox_Fe,NUTS_0,NUTS_1,NUTS_2,NUTS_3,SURVEY_DATE.1,LC0_Desc,LC1_Desc,LU1_Desc
10374,48063424,"POLYGON ((17.35658319098807 53.70162514515486,...",PL,PL6,PL63,PL637,53.701584,17.357216,0,6/9/2018,...,NaN,NaN,PL,PL6,PL63,PL637,06-09-18,Woodland,Pine dominated mixed woodland,Semi-natural and natural areas not in use
3337,47401682,"POLYGON ((14.74653441594476 38.10151242796173,...",IT,ITG,ITG1,ITG13,38.101550,14.746640,0,18/06/18,...,NaN,NaN,IT,ITG,ITG1,ITG13,18-06-18,Grassland,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...
11524,51103486,"POLYGON ((22.06168121842496 53.88767746384045,...",PL,PL6,PL62,PL623,53.887647,22.062244,0,28/05/18,...,NaN,NaN,PL,PL6,PL62,PL623,28-05-18,Grassland,Grassland without tree/shrub cover,Agriculture (excluding fallow land and kitchen...


Remove unecessary features

In [5]:
indexation = ["POINT_ID", "geometry", "SURVEY_DATE", "TH_LAT", "TH_LONG", "Depth"]
land_cover = ["LC0_Desc", "LC1_Desc"]
groundtruth = [ "pH_CaCl2", "pH_H2O", "EC", "OC", "CaCO3", "P", "N", "K", "Ox_Al", "Ox_Fe"]

df = df[indexation + land_cover + groundtruth]
df.sample(3)

,POINT_ID,geometry,SURVEY_DATE,TH_LAT,TH_LONG,Depth,LC0_Desc,LC1_Desc,pH_CaCl2,pH_H2O,EC,OC,CaCO3,P,N,K,Ox_Al,Ox_Fe
9068,55162128,"POLYGON ((24.28891502145054 41.23202876665795,...",13/07/18,41.231939,24.289519,0-20 cm,Woodland,Broadleaved woodland,5.7,6.26,15.27,13,3,17.8,1.1,100.6,NaN,NaN
965,28321780,POLYGON ((-6.889795863141356 37.61017943265134...,15/06/18,37.610279,-6.889221,0-20 cm,Woodland,Broadleaved woodland,4.0,4.85,4.82,7.7,3,< LOD,1.3,51.9,NaN,NaN
10321,51664234,"POLYGON ((25.41537962221738 60.37425085586095,...",13/08/18,60.374174,25.416320,0-20 cm,Woodland,Pine dominated mixed woodland,3.6,4.43,7.36,69.9,1,13.3,3.6,89.8,NaN,NaN


Convert '< LOD' to nan

In [6]:
df.loc[:, groundtruth] = df[groundtruth].apply(
    lambda x: pd.to_numeric(x, errors="coerce")
)
df[groundtruth] = df[groundtruth].astype(float)

Compute pH mean

In [7]:
df["pH"] = df[["pH_CaCl2", "pH_H2O"]].mean(axis=1).round(3)
df = df.drop(columns=["pH_CaCl2", "pH_H2O"], axis=1)

In [8]:
df.sample(10)

,POINT_ID,geometry,SURVEY_DATE,TH_LAT,TH_LONG,Depth,LC0_Desc,LC1_Desc,EC,OC,CaCO3,P,N,K,Ox_Al,Ox_Fe,pH
4538,51223770,"POLYGON ((23.03686167310365 56.37633061889685,...",27/04/18,56.376290,23.037253,0-20 cm,Cropland,Common wheat,18.11,16.8,18.0,30.1,1.6,113.5,NaN,NaN,7.345
8520,41423356,"POLYGON ((7.315011032789062 53.28227368792784,...",11/6/2018,53.282303,7.315805,0-20 cm,Grassland,Grassland without tree/shrub cover,19.20,75.2,NaN,70.8,6.7,331.2,NaN,NaN,4.885
5656,51464814,"POLYGON ((27.90909151767875 65.47293018534462,...",13/08/18,65.472827,27.909976,0-20 cm,Woodland,Other mixed woodland,4.42,40.5,NaN,30.8,2.2,43.5,NaN,NaN,4.305
11289,40322632,"POLYGON ((6.219879048227392 46.73514762076917,...",7/8/2018,46.735180,6.220555,0-20 cm,Woodland,Spruce dominated coniferous woodland,15.98,113.5,1.0,NaN,6.3,160.3,NaN,NaN,5.340
12803,49844400,"POLYGON ((22.74832515808303 62.15937567896084,...",4/7/2018,62.159255,22.749278,0-20 cm,Woodland,Pine dominated coniferous woodland,9.57,461.3,3.0,19.0,13.8,114.9,NaN,NaN,3.325
823,31203468,POLYGON ((-8.089992121260693 52.96359822698882...,23/08/18,52.963706,-8.089257,0-20 cm,Cropland,Temporary grassland,14.98,29.0,8.0,18.6,3.4,73.8,NaN,NaN,7.115
9582,43262834,"POLYGON ((10.06709406268337 48.61936478606255,...",28/05/18,48.619365,10.067792,0-20 cm,Grassland,Grassland without tree/shrub cover,14.12,16.4,1.0,35.7,1.8,208.2,NaN,NaN,6.890
6341,49064496,"POLYGON ((21.60145387511463 63.13048172966707,...",6/7/2018,63.130410,21.601729,0-20 cm,Woodland,Other mixed woodland,6.83,60.0,1.0,26.4,2.6,84.0,NaN,NaN,3.995
4600,45604012,"POLYGON ((14.16921365070466 59.14578648232117,...",17/09/18,59.145759,14.170087,0-20 cm,Woodland,Pine dominated coniferous woodland,28.20,269.2,1.0,36.1,13.2,147.6,NaN,NaN,3.865
6194,57021478,"POLYGON ((25.10029301548668 35.13356036715209,...",7/4/2018,35.133488,25.100844,0-20 cm,Cropland,Olive groves,16.54,12.7,168.0,NaN,1.3,171.3,1.2,1.6,7.690


In [9]:
df.to_csv('./Manuel/LUCAS.csv', index=False)